In [1]:
# Import modules
import logging
import sys
import time
import paths
print(paths.ROOT_FOLDER)
from caching_functions import generate_cache_parameters_from_command_line_argument
from parameters import Parameters
from handling_functions import check_script_call
from model import Model

C:\Users\colon\OneDrive\WorldBank\Model\disrupt-sc


ModuleNotFoundError: No module named 'code.paths'; 'code' is not a package

In [32]:
from parameters import Parameters

ModuleNotFoundError: No module named 'yaml'

In [37]:
sys.path

['C:\\Users\\colon\\OneDrive\\WorldBank\\Model\\disrupt-sc\\test',
 'C:\\Users\\colon\\OneDrive\\WorldBank\\Model\\disrupt-sc',
 'C:\\Users\\colon\\Anaconda3\\envs\\geo_env\\python39.zip',
 'C:\\Users\\colon\\Anaconda3\\envs\\geo_env\\DLLs',
 'C:\\Users\\colon\\Anaconda3\\envs\\geo_env\\lib',
 'C:\\Users\\colon\\Anaconda3\\envs\\geo_env',
 '',
 'C:\\Users\\colon\\Anaconda3\\envs\\geo_env\\lib\\site-packages',
 'C:\\Users\\colon\\Anaconda3\\envs\\geo_env\\lib\\site-packages\\win32',
 'C:\\Users\\colon\\Anaconda3\\envs\\geo_env\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\colon\\Anaconda3\\envs\\geo_env\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\colon\\Anaconda3\\envs\\geo_env\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\colon\\.ipython',
 WindowsPath('C:/Users/colon/OneDrive/WorldBank/Model/disrupt-sc'),
 WindowsPath('C:/Users/colon/OneDrive/WorldBank/Model/disrupt-sc'),
 WindowsPath('C:/Users/colon/OneDrive/WorldBank/Model/disrupt-sc/cide'),
 WindowsPath('C:/Users/colon

In [10]:
import os 
#if you want to know current working dir
os.getcwd()

'C:\\Users\\colon\\OneDrive\\WorldBank\\Model\\disrupt-sc\\test'